In [193]:
import pandas as pd
import numpy as np

In [194]:
DIR = '../dataset/'
DATASET = 'Nutrition_Physical_Activity_and_Obesity_Data.csv'
df = pd.read_csv(DIR + DATASET)

In [195]:
df.columns = df.columns.str.strip()

for col in df.columns:
    if df[col].dtype != np.dtypes.Int64DType:
        continue

    df[col] = df[col].str.rstrip()
    df[col] = df[col].str.rstrip()

df['Data_Value'] = df['Data_Value'].replace('', -1)
df['Data_Value'] = df['Data_Value'].astype(float)

In [196]:
states = df['LocationDesc'].unique()
years = list(range(2010, 2023))*len(states)
states = np.repeat(states, 13)
new_df = pd.DataFrame({'state': states, 'year': years})
new_df

,state,year
0,Alaska,2010
1,Alaska,2011
2,Alaska,2012
3,Alaska,2013
4,Alaska,2014
...,...,...
710,Oregon,2018
711,Oregon,2019
712,Oregon,2020
713,Oregon,2021


In [197]:
#df.to_csv('../udataset/' + DATASET, index=False)

def get_label_function(desired_question, df, desired_col):

    def get_attr_column(row):
        to_return = df[(df['Question'] == desired_question) & (df['YearEnd'] == row['year']) & (df['LocationDesc'] == row['state']) & (df['Data_Value'] != -1) & (df['StratificationCategory1'] == 'Total')][desired_col].mean()
        
        return to_return

    return get_attr_column

In [198]:
desired_questions = ['Percent of adults aged 18 years and older who have obesity', 'Percent of adults aged 18 years and older who have an overweight classification', 'Percent of adults who engage in no leisure-time physical activity']
column_names = ['obesity_adult_rate', 'overweight_adult_rate', 'adults_no_leisure-time_physical_activity']

In [199]:
row = df[(df['Question'] == 'Percent of adults aged 18 years and older who have obesity') & (df['YearEnd'] == 2012) & (df['LocationDesc'] == 'Alaska') & (df['Data_Value'] != -1)]['Data_Value']
print(row.mean())
for index, question in enumerate(desired_questions):
    new_df[column_names[index]] = new_df.apply(get_label_function(question, df, 'Data_Value'), axis=1)

24.929629629629627


In [200]:
new_df

,state,year,obesity_adult_rate,overweight_adult_rate,adults_no_leisure-time_physical_activity
0,Alaska,2010,NaN,NaN,NaN
1,Alaska,2011,27.4,38.9,22.0
2,Alaska,2012,25.7,39.1,18.5
3,Alaska,2013,28.4,37.7,22.3
4,Alaska,2014,29.7,35.1,19.2
...,...,...,...,...,...
710,Oregon,2018,29.9,34.2,19.3
711,Oregon,2019,29.0,34.9,23.9
712,Oregon,2020,28.1,36.2,18.5
713,Oregon,2021,30.4,34.7,19.9


In [204]:
for col in new_df.columns[2:]:
    new_df[col] = (new_df[col] - new_df[col].min())/(new_df[col].max() - new_df[col].min())

In [205]:
NEWNAME = 'health_data.csv'
new_df.to_csv('../udataset/' + NEWNAME, index=False)